# Analysis

## Towards better data management and plotting: Pandas and Seaborn

Pandas allows the creation of `DataFrames` that neatly capture multi-dimensional arrays that can be more easily understood and visualised. 

Seaborn neatly plots tidy (“long-form”) dataframes where each column is a variable and each row is an observation. [Tidy Data in Python](https://www.jeannicholashould.com/tidy-data-in-python.html).

Pandas has some awesome ways to load data.

E.g. read from an excel spreadsheet (can even reference an online document)

[`df = pd.read_excel(...)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html)

E.g. save to a "comma-separated value" file

[`df.to_csv("file name.csv")`](https://dev.pandas.io/docs/user_guide/io.html#io-store-in-csv)


In [ ]:
import seaborn as sns
import pandas as pd

# seaborn has some built-in datasets
dots = sns.load_dataset("dots")
# display a pandas dataframe, loaded from data
print(type(dots))
dots

### Converting between long and wide data
[Reshaping](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)


- to wide:
  - [`pivot`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html#pandas.DataFrame.pivot) or [`pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table)

  <img src="https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_pivot.png" width="500px">

  - [`stack`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html?highlight=unstack#pandas.DataFrame.stack)

  <img src="https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_stack.png" width="500px">

- to long
  - [`melt`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html#pandas.DataFrame.melt)

  <img src="https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_melt.png" width="500px">

  - [`unstack`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html?highlight=unstack#pandas.DataFrame.unstack)

  <img src="https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_unstack.png" width="500px">




#### Example

In [ ]:
wide_df = dots.pivot_table(index='time', columns=['align','coherence','choice'], values=['firing_rate'])
wide_df

In [ ]:
# first make a copy so we can manipulate it before converting without affecting the original
time_as_col = wide_df.copy()
# make the index an explicit column (melt tends to ignore index)
time_as_col['time'] = time_as_col.index
# reshape
long_df = time_as_col.melt(id_vars=['time'], value_vars=['firing_rate'], value_name='firing_rate')
# show a sample
print(long_df.head())
print(f"(rows, columns) = {long_df.shape}")
# clean up by removing NaNs and removing the NaN (None) column
long_df = long_df.dropna().drop(columns=[None])
long_df

### Plotting long and wide data

##### Plot wide data using Pandas and Matplotlib

In [ ]:
#simplest version (with 2 columns)
fig, ax = plt.subplots(1, 2, sharey=True)
for _ax, align in zip(ax,['dots','sacc']):
  dropped_df = wide_df['firing_rate',align].dropna(how='all')
  style=['-',':']*int(dropped_df.columns.get_level_values('choice').size/2)
  dropped_df.plot(style=style, ax=_ax, cmap='viridis', legend=align=='sacc')
  _ax

# annotate
ax[0].set_ylabel('firing_rate')
ax[1].legend(title=ax[1].get_legend().get_title().get_text(), frameon=False,
             loc='upper left', bbox_to_anchor=(1,1))
# clean up
for _ax in ax:
  for s,spine in _ax.spines.items():
    if s=='top' or s=='right':
      spine.set_visible(False)

more control over the plotting

what we want is 
- each `align` to be on different axes
- each ` choice` to be a different line width
- each `coherence` to be a different colour

we iterate over slices of the `pd.DataFrame` (where each slice is a `pd.Series` object with a flattened index (`name` in the for loop) 

`name=(<align>,<coherence>,<choice>)` and plot these slices.



In [ ]:
import matplotlib
from matplotlib import cm

# Change ax to plot on by 'align'
# Get all alignments (set is used to only have unique values)
align = set(wide_df.columns.get_level_values('align'))
ncols = len(align)
fig, ax = plt.subplots(1,ncols, sharey=True)
# zip combines 2 iterables (like list or set) such that 
#   zip([a,b],[1,2]) == [(a,1),(b,2)]
#   (zip returns anobject generator, but wrap list around it for above)
# when we pass zip to dict, it converts the list of tuples () to key:value pairs
align_to_ax = dict(zip(sorted(align),ax))

# Change linewidth by 'choice' - `[::-1]` reverses a list
choices = sorted(set(wide_df.columns.get_level_values('choice')))[::-1]
lw_min = 1
lw_scale = 5
choice_to_lw = dict(zip(choices,
                        np.arange(lw_min,len(choices)*lw_scale+lw_min,lw_scale)))

# Change color by 'coherence'
coherence = sorted(set(wide_df.columns.get_level_values('coherence')))
norm = matplotlib.colors.Normalize(vmin=min(coherence), vmax=max(coherence))
# like list comphrehension, we an do dict comprehension
coh_to_color = {coh:cm.magma_r(norm(coh)) for coh in coherence}

# iterate over slices of the dataframe
for name, series in wide_df['firing_rate'].iteritems():
  # print(name)
  series.dropna(how='all').plot(ax=align_to_ax[name[0]], 
              lw=float(choice_to_lw[name[2]]), 
              c=coh_to_color[name[1]],legend=False)
  
# annotate
for align,_ax in align_to_ax.items():
  _ax.set_title(f'align = {align}')
ax[0].set_ylabel('firing_rate')

from matplotlib.lines import Line2D
# create legend for choice - list of tuples (Line, label)
leg_choice = [(Line2D([],[],c='k', lw=choice_to_lw[choice]), choice) for choice in choices][::-1]

# *zip(*leg_choice) works by converting [(line1,label1), (line2, label2)] to 
#   [[line1, line2], [label1, label2]] - this is zip(*leg_choice)
# legend expects the first argument to be lines and the 2nd labels, so we spread
#   the list of 2 lists [[],[]] using the * operator to only have 2 lists [],[].
leg = ax[1].legend(*zip(*leg_choice), title='choice',
                   loc='lower left', bbox_to_anchor=(1,-0.1), ncol=2, frameon=False)
# use the existing dict, which can be problematic if ordering matters
#   (see comment below)
leg_coh = [(Line2D([],[],c=c), coh) for coh,c in coh_to_color.items()]
# note that dicts are unsorted
# we can use something like `from collections import OrderedDict` and some 
#   settings to maintaing an ordering.
# here we use an anonymous function on the leg_coh list
leg_coh = sorted(leg_coh, key=lambda linelabel: (linelabel[1],linelabel[0]))
# we lose the previous legend when we call legend() again
ax[1].legend(*zip(*leg_coh), title='coherence',
             loc='upper left', bbox_to_anchor=(1,1.1), frameon=False)
# but we can have both by explicitly adding the object back
ax[1].add_artist(leg)

# clean up axes
for _ax in ax:
  for s,spine in _ax.spines.items():
    if s=='top' or s=='right':
      spine.set_visible(False)


we learnt some **new** Python above
> `tuple` - an immutable iterable akin to a list
  \
  *immutable* means you cannot change the data after you have created the object. You can reassign the name, though. 
  \
  this will fail
  ```
  t = (1,5,10)
  t[0] = 2
  ```
  hint: you can use tuples as default arguments in methods directly (instead of using `None` like with `list`s and `dict`s

---
> `set` - an iterable that will only have unique values (no duplicates)
   \
   sets are created with `set()` or `{}` and are **unsorted** but very fast - O(1) lookup
   \
   instead of `.append` you use `.add`
   \
   ```
   s = {1,10,10,100,100,100,1000,1000,1000,1000}
   print(s) # {1000, 1, 10, 100}
   ```

---
> `zip` - combines multiple iterables (e.g. `list`, `array`, `set`, `tuple`, `str`)
  \
  ```
  chars = 'abcde'
  nums = range(1,10)
  zipped = zip(s, range(10))
  print(zipped)       # <zip object at 0x7ff505aabc88>
  print(list(zipped)) # [('a', 0), ('b', 1), ('c', 2), ('d', 3), ('e', 4)]
  print(*zipped)      # ('a', 0), ('b', 1), ('c', 2), ('d', 3), ('e', 4)
  ```
  iterables are truncated to the shortest one

---
> `*` - spread operator
  \
  we have used `*` in methods as a **rest operator** to 'collect' other arguments (and `**` to collect other *keyword* arguments). 
  \
  here, the operator 'flattens' an iterable. This can be useful for passing to methods, for copying data, and can be combined with `zip` to adjust the format of 2 related lists.
  \
  ```
  unzipped = zip(*zipped) # unzip and get original 2 lists
  chars, nums, *rest = unzipped # note *rest is recent in Python
  print(f"chars={chars}, nums={nums}, rest={rest}")
  ```

---
> `lambda` - anonymous functions
  \
  when we're feeling lazy$^1$ and don't want to name a function explicitly to implement a "simple" expression. Actually it can be named, go figure 🤷‍♂️.
  \
  ```
  anon = lambda x: x*2
  for i in range(10):
    print(f"{i} - {anon(2):>2g"})
  ```
  we passed an annoymous function to the `sorted` function (as part of the `key` param) in order to sort by `tuple`'s 2nd value (index `1`). The `key` param expects a single arg so we access the 2nd value of the passed `tuple`. We (silently) return `(x[1],x[0])` but it would sufficent in this case to just return `x[1]` because we don't care about additional sorting.
  \
  [read more on sorting by value](https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value)


1. efficient


#### Plot long data using Seaborn

In [ ]:
# try changing style and context to see how it affects the plot
sns.set(style="ticks", context="talk")

# Define a palette to ensure that colors will be
# shared across the facets
palette = dict(zip(dots.coherence.unique(),
                   sns.color_palette("rocket_r", 6)))

# Plot the lines on two facets
sns.relplot(x="time", y="firing_rate",
            hue="coherence", size="choice", col="align",
            size_order=["T1", "T2"], palette=palette,
            height=5, aspect=.75, facet_kws=dict(sharex=False),
            kind="line", legend="full", data=dots)

## Analysing someone else's code



- can you make sense of the code below?
- what happens when you run it?
- are there inconsistencies in the variable names, function calls, or comments?

In [ ]:
import cmath
import time
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import uniform, gamma, seed
from scipy.sparse import csr_matrix

seed(86664)

## Parameter values
#   Time
dt = 0.5  # ms
timeEnd = 5000  # ms
T = int(np.ceil(timeEnd / dt))  # Number of time_array points

#   Neurons
n = 1000
inh_ratio = 0.2
n_inh = int(n * inh_ratio)
n_exc = int(n * (1 - inh_ratio))
k_inh = np.append(np.ones(n_inh, dtype='bool'), np.zeros(n_exc, dtype='bool'))
# uniform(size=n) < 0.2  # 20 % inhibitory
k_exc = np.logical_not(k_inh)  # opposite -> could also use (k_inh == False)

a = k_inh.choose(0.02, 0.1)  # inhibitory cells a=0.1, excitatory a=0.02
b = 0.2
c = -65
d = k_inh.choose(8, 2)
tau_s = 10  # decay of synapses (ms)
tau_d = 500  # synaptic depression (ms)
stp_u = 0.5  # STP parameter
ErevExc = 0.0
ErevInh = -75.0

# Input
Itime = np.array([100, timeEnd-500]) / dt
n_in = 100  # number of synaptic inputs
w_in = 0.2
pConnection = 0.01  # connection probability 'spontaneous activity'
seed(int(pConnection * 100))
C = uniform(size=(n, n_in)) < pConnection
W_in = C.choose(0, w_in)
f_rate = 0.010  # ms^-1)
prate = f_rate * dt

# Recurrent Input
w_avg = 0.002  # average recurrent weight
pConnectionRecurrent = 0.2  # connection probability
W = np.zeros((n, n))
C = uniform(size=(n, n))
idx = np.nonzero(C < pConnectionRecurrent)
g_sc = 0.002
g_sh = w_avg / g_sc
W[idx] = gamma(g_sh, g_sc, size=idx[0].size)  # gamma distributed array (shape,scale,size)
scaleEI = 2  # weights from I->E different
W[np.ix_(k_exc, k_inh)] *= scaleEI
W = csr_matrix(W)  # make row sparse

## Memory
time_array = np.arange(1, timeEnd + 1, dt)
v = np.zeros((T, n))
u = np.zeros((T, n))
v[0] = -70  # resting potential
# Leak channels
EL = -70
gL = 0.0001
u[0] = -14  # steady state

S_in = np.zeros(n_in)  # synaptic input from external source
s = np.zeros(n)  # recurrent synapses
E_in = np.zeros(n_in)
H_in = np.ones(n_in)  # synaptic depression term
H = np.ones(n)  # synaptic depression term
prevSpike = -np.infty * np.ones(n_in)
prevSpikeRec = -np.infty * np.ones(n)
E = k_inh.choose(ErevExc, ErevInh)

r = uniform(size=n_in)
# v = (r<p).choose(b,a)

for t in np.arange(T-1):
    if t * dt % 1000 == 0:
        print(f'{t * dt / 1000}  s simulated.')
    if Itime[0] < t < Itime[1]:
        I = 0                           # background current
        # Get input spikes (each neuron receives ~10% of input spikes)
        P = uniform(size=n_in) < prate  # background input
        tmp = np.exp(dt * (prevSpike[P] - t) / tau_d)
        H_in[P] = 1 - (1 + (stp_u - 1) * H_in[P]) * tmp
        prevSpike[P] = t
    else:
        I = 2
        P = 0

    # Calculate input current (all excitatory input)
    S_in = (1 - dt / tau_s) * S_in + P * H_in
    I += W_in.dot(S_in * E_in) - (W_in.dot(S_in) * v[t])

    # handle all neurons
    fired = v[t] >= 35
    tmp = np.exp(dt * (prevSpikeRec[fired] - t) / tau_d)
    H[fired] = 1 - (1 + (stp_u - 1) * H[fired]) * tmp
    prevSpikeRec[fired] = t

    # recurrent input
    s = (1 - dt / tau_s) * s + fired
    Irec = W * s * (E - v[t])
    I += Irec

    # Update ODE
    dv = (0.04 * v[t] + 5) * v[t] + 140 - u[t]
    v[t + 1] = v[t] + (dv + I) * dt
    du = a * (b * v[t] - u[t])
    u[t + 1] = u[t] + du * dt
    # Spike!
    v[t][fired] = 35
    v[t + 1][fired] = c
    u[t + 1][fired] = u[t][fired] + d[fired]


#### Tasks

##### **Task 1**
Write the system of equations that govern the neurons' behaviour

Extra questions:

- what does `np.ix_` do?
- what does `csr_matrix` do? Is the comment useful?



**Space for solution**

[EDIT ME]

eq.1: $\frac{dv}{dt} = ... \tag{neuronal voltage}$

...

##### **Task 2**: plotting
1. plot the **membrane potential** over time for the excitatory "pyramidal cell" neurons (**PC**) in red and the inhibitory "interneuron" neurons (**IN**) in blue. 
1. use a **raster plot** to show the spikes times (x-axis) for each neuron (y-axis). 

  ![example](https://upload.wikimedia.org/wikipedia/en/5/58/Sample_raster_plot_from_Brian_neural_network_simulator.jpg)

1. plot a **histogram** of firing rates for each population

Extra questions:
- are PCs always excitatory? If not, under what condition(s)?
- are INs always inhibitory? If not, under what condition(s)?



In [ ]:
#@title **Task 2** Solution { run: "auto", vertical-output: true, display-mode: "form" }
time_width = 530 #@param {type:"slider", min:100, max:1000, step:50}
# Plot
fig, ax = plt.subplots(3, 1, sharex=True, figsize=(4,8),
                       gridspec_kw={'height_ratios':[5,10,7]})

# voltage vs time
ax[0].plot(time_array, v[:, k_exc], 'r', alpha=0.1, lw=0.1)
ax[0].plot(time_array, v[:, k_inh], 'b', alpha=0.1, lw=0.1)
ax[0].plot(time_array, np.mean(v[:, k_exc],axis=1), 'r',  label='PC')
ax[0].plot(time_array, np.mean(v[:, k_inh],axis=1), 'b',  label='IN')


# get time (rows) and neuron index (columns) of spikes from v
tspk, nspk = np.nonzero(v >= 35)
idx_i = np.in1d(nspk, np.nonzero(k_inh)[0])  # inh spikes
idx_e = np.logical_not(idx_i)                # exc spikes

# nrn index vs time
ax[1].plot(tspk[idx_e] * dt, nspk[idx_e], 'r.', label='Exc.', markersize=1)
ax[1].plot(tspk[idx_i] * dt, nspk[idx_i], 'b.', label='Inh.', markersize=1)

# plot histogram of firing rate
bins = int(timeEnd / time_width)
# plot histogram
for i in range(0, bins - 1):
    spikes_in_bin_indices = (tspk > time_width * i) & (tspk < time_width * (i + 1))
    if np.any(spikes_in_bin_indices):
        spikes_in_bin = tspk[spikes_in_bin_indices]
        inh_spikes_in_bin = tspk[(idx_i) & (spikes_in_bin_indices)]
        exc_spikes_in_bin = tspk[(idx_e) & (spikes_in_bin_indices)]
        exc = ax[2].bar((time_width * i), len(exc_spikes_in_bin) / n_exc, 
                width=0.5*time_width,align='edge',color='r')
        inh = ax[2].bar((time_width * i)+0.5*time_width, len(inh_spikes_in_bin) / n_inh, 
                width=0.5*time_width,align='edge',color='b')
        avg = ax[2].bar((time_width * i), 0.0, bottom=len(spikes_in_bin) / n,
                width=time_width, align='edge',color='k', alpha=1,edgecolor='k')
    else:
        ax[2].bar(time_width * i, 0, width=time_width)

# annotate
ax[0].set_ylabel('Membrane voltage \n(mV)')
ax[1].set_ylabel('Neuron index \n(#)')
ax[2].set_ylabel('Firing rate \n(Hz)')

ax[-1].set_xlim(0, timeEnd)
ax[-1].set_xlabel('Time (ms)')
ax[0].legend(loc='upper left', bbox_to_anchor=(1,1), frameon=False)
ax[2].legend([avg], ['$\mu (PC + IN)$'], loc='best', frameon=False)

# clean
sns.despine() # remove top + right borders

##### **Task 3**: plotting with Seaborn

1. create a *long form* `DataFrame` with "Neuron Index" "Time" and "Type" columns ("Time" is the spike time) and each row is an observation.

1. plot a raster plot

1. plot a raster plot with *kernel density estimates* (run, but don't look at, the solution to see what I mean)

In [ ]:
import seaborn as sns

In [ ]:
#@title **Task 3.1** Solution { run: "auto", vertical-output: true, display-mode: "form" }
import pandas as pd
spikes = pd.DataFrame({"Neuron Index":nspk, "Time":tspk*dt})
spikes["Type"] = spikes["Time"]
spikes.loc[idx_i,"Type"] = "IN"
spikes.loc[idx_e,"Type"] = "PC"
spikes

In [ ]:
#@title **Task 3.2** Solution { run: "auto", vertical-output: true, display-mode: "form" }
sns.scatterplot(x="Time", y="Neuron Index",
             hue="Type",  palette="Dark2", s=2, ec='None',
             data=spikes)
plt.xlim(0, timeEnd)
plt.ylim(0, n)
plt.title("Raster plot")
plt.legend(loc=(1,0.5), frameon=False)

In [ ]:
#@title **Task 3.3** Solution { run: "auto", vertical-output: true, display-mode: "form" }
g = sns.jointplot(x="Time", y="Neuron Index", s=0.2, data=spikes)
g.ax_joint.axhline(y=200, lw=2, c='k')
g.ax_joint.set_xlim(0, timeEnd)

### **Homework/Project 3** Implement a balanced spiking neural network with 1000 neurons, 20% interneurons, 2% general connectivity (including recurrent). Implement $AMPA$ and $GABA_A$ synapses, and a *low* background current. 

Build from scratch or use [Brian2](https://brian2.readthedocs.io/en/stable/) (or another simulator!)

1. How do you scale the synaptics conductances with the number of neurons? 
1. Change the $AMPA$ and $GABA_A$ time constants, is the scaling rule still the same?
1. Implement $NMDA$ and/or $GABA_B$ synapses. How has the scaling rule changed now?
1. Restrict interneuron to only connect with neurons within 10 mm. Restrict pyramidal neurons to only connect with neurons at least 50 mm away. *hint: you may need to set up a virtual "grid" (2D/3D) on which to place neurons*.

